In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from google.colab.patches import cv2_imshow

model_path = '/content/fire_det.h5'
model = load_model(model_path)

video_path = '/content/sample21.mp4'
cap = cv2.VideoCapture(video_path)


total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)

time_interval = 1 / fps

class_counts = {'s0': 0, 's1': 0, 's2': 0, 's3': 0}

current_frame = 0
while current_frame < total_frames:

    cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)

    ret, frame = cap.read()

    if not ret:
        break

    resized_frame = cv2.resize(frame, (224, 224))
    normalized_frame = resized_frame / 255.0

    input_frame = np.expand_dims(normalized_frame, axis=0)
    predictions = model.predict(input_frame)

    predicted_class = np.argmax(predictions, axis=1)[0]

    class_counts[f's{predicted_class}'] += 1

    current_frame += int(fps)

final_fire_stage = max(class_counts, key=class_counts.get)


cap = cv2.VideoCapture(video_path)
ret, final_frame = cap.read()
cap.release()

# Display the final result with the thumbnail
cv2.putText(final_frame, f"Final Fire Stage: {final_fire_stage}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
cv2_imshow(final_frame)
cv2.waitKey(0)

cap.release()
cv2.destroyAllWindows()
